# Plots of SFH for Outshined/not Outshined Galaxies

In [ ]:
import astropy.units as u
import numpy as np
import os
from astropy.table import Table
import copy
os.environ["PIXEDFIT_HOME"] = "/nvme/scratch/work/tharvey/piXedfit/"
from EXPANSE import ResolvedGalaxy, ResolvedGalaxies
from matplotlib import pyplot as plt
import matplotlib.patheffects as pe
import glob
from scipy import signal
from scipy.interpolate import interp1d
from scipy.stats import binned_statistic
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.cm as cm
from cmasher import take_cmap_colors

# Change dpi to make plots larger

plt.rcParams["figure.dpi"] = 100

# Disable tex in matplotlib

plt.rcParams["text.usetex"] = False
%matplotlib inline

### Initial Loading of the Galaxy Objects

In [ ]:
galaxies = ResolvedGalaxies(
    ResolvedGalaxy.init_all_field_from_h5("JOF_psfmatched", n_jobs = 6)
)

Choosing a sample to plot. 

In [ ]:
ids = [10536, 9634, 10045, 15021, 5371]
ids = [str(i) for i in ids]


test_galaxies = galaxies.filter_IDs(ids)

ids = [10130, 12816, 2994, 845, 615, 7687]
ids = [str(i) for i in ids]

control_galaxies = galaxies.filter_IDs(ids)

Plotting the SFH plot for the different models.

In [ ]:

fig = plt.figure(figsize=(4, 10), dpi = 200, tight_layout = True, facecolor = 'white')

axes = fig.subplots(len(control_galaxies), 1, sharex=True)

resolved_runs = ['CNST_SFH_RESOLVED', 'CNST_SFH_RESOLVED_NOMIN', 'CNST_SFH_RESOLVED_VORONOI']
resolved_run_names = ['Resolved (piXedfit)', 'Resolved (piXedfit, no min)', 'Resolved (voronoi)']
# nice color palette with distinct colors

resolved_cmap = 'cmr.apple'
resolved_colors = take_cmap_colors(resolved_cmap, len(resolved_runs), cmap_range=(0, 0.8))
resolved_colors = ['mediumseagreen', 'gold', 'orangered']
resolved_color_dict = dict(zip(resolved_runs, resolved_colors))

integrated_runs = ['photoz_lognorm', 'photoz_delayed', 'photoz_continuity', 'photoz_cnst', 'photoz_continuity_bursty']
integrated_run_names = ['Lognorm SFH', r'Delayed-$\tau$ SFH', 'Continuity SFH', 'Constant SFH', 'Bursty SFH']
# uniuqe cmap - not viridis
integrated_cmap = 'cmr.guppy'
# get colors from cmap
cmap = plt.get_cmap(integrated_cmap)
integrated_colors = [cmap(i) for i in np.linspace(0, 1, len(integrated_runs))]
integrated_color_dict = dict(zip(integrated_runs, integrated_colors))
resolved_run = "CNST_SFH_RESOLVED_VORONOI"
print(len(test_galaxies), len(axes))
for i, galaxy in enumerate(control_galaxies):
    print(i)
    ax = axes[i]

    for integrated_run, integrated_color in zip(integrated_runs, integrated_colors):
        _, _ = galaxy.plot_bagpipes_sfh(run_name = integrated_run, axes = ax, bins_to_show=['TOTAL_BIN'], fig = fig, marker_colors = [integrated_color], time_unit='yr', linestyle = 'dashdot')
    
    
    for resolved_runa, resolved_color in zip(resolved_runs, resolved_colors):
        try:
            _, _ = galaxy.plot_bagpipes_sfh(run_name = resolved_runa, axes = ax, bins_to_show=['RESOLVED'], fig = fig, resolved_color=resolved_color, time_unit='yr', zorder=10)
        except:
            pass

    # Delete legend
    ax.get_legend().remove()

    ax.set_xscale('log')
    ax.set_xlabel('')
    # Label with galaxy ID and redshift

    ax.text(0.05, 0.95, f'{galaxy.galaxy_id}\nz={galaxy.redshift:.2f}\n$N_{{bins}}={galaxy.get_number_of_bins(binmap_type="voronoi")}$', transform=ax.transAxes, ha='left', va='top', zorder=30, fontsize=8, path_effects=[pe.withStroke(linewidth=1, foreground='white')])
    
    # Write resolved stellar mass 

    # Get min, max mass from table
    min_mass = (1e10, 0, 0)
    max_mass = (0, 0, 0)
    for run in galaxy.sed_fitting_table['bagpipes'].keys():
        if run == 'photoz_dpl':
            continue
        
        if 'TOTAL_BIN' in galaxy.sed_fitting_table['bagpipes'][run]['#ID']:
            mass = galaxy.sed_fitting_table['bagpipes'][run]['stellar_mass_50'][0]
            upper = galaxy.sed_fitting_table['bagpipes'][run]['stellar_mass_84'][0] - mass
            lower = mass - galaxy.sed_fitting_table['bagpipes'][run]['stellar_mass_16'][0]

            if mass > max_mass[0]:
                max_mass = (mass, upper, lower, run)
            elif mass < min_mass[0]:
                min_mass = (mass, upper, lower, run)

    ax.text(0.97, 0.75, f'$\log_{{10}}(M_{{\star, \mathrm{{integrated, max}}}}) = {max_mass[0]:.2f}^{{+{max_mass[1]:.2f}}}_{{-{max_mass[2]:.2f}}}$', transform=ax.transAxes, ha='right', va='top', fontsize=8, path_effects=[pe.withStroke(linewidth=1, foreground='white')], color=integrated_color_dict[max_mass[3]],  zorder=30)
    ax.text(0.97, 0.85, f'$\log_{{10}}(M_{{\star, \mathrm{{integrated, min}}}}) = {min_mass[0]:.2f}^{{+{min_mass[1]:.2f}}}_{{-{min_mass[2]:.2f}}}$', transform=ax.transAxes, ha='right', va='top', fontsize=8, path_effects=[pe.withStroke(linewidth=1, foreground='white')], color=integrated_color_dict[min_mass[3]],  zorder=30)


    val = galaxy.resolved_mass[resolved_run][1]
    upper = galaxy.resolved_mass[resolved_run][2] - val
    lower = val - galaxy.resolved_mass[resolved_run][0]

    ax.text(0.97, 0.95, f'$\log_{{10}}(M_{{\star, \mathrm{{resolved}}}}) = {val:.2f}^{{+{upper:.2f}}}_{{-{lower:.2f}}}$', transform=ax.transAxes, ha='right', va='top', fontsize=8, path_effects=[pe.withStroke(linewidth=1, foreground='white')], color=resolved_color_dict[resolved_run],  zorder=30)

    ax.set_xlim(1e6, 5e8)

# Add a dummy legend with SFHs

axes[-1].set_xlabel('Lookback Time [yr]')

points = []
for pos, (resolved_run, resolved_color) in enumerate(zip(resolved_run_names, resolved_colors)):
    point = ax.plot([], [], color = resolved_color, label = resolved_run)
    points.append(point[0])

for pos, (integrated_run, integrated_color) in enumerate(zip(integrated_run_names, integrated_colors)):
    point = ax.plot([], [], color = integrated_color, label = integrated_run, linestyle = 'dashdot')
    points.append(point[0])

fig.legend(handles = points, loc = 'lower center', fontsize = 8, title = 'SFH Type', title_fontsize = 8, markerscale = 0.7, labelspacing=0.1, ncol=2, bbox_to_anchor=(0.5, -0.07))

fig.savefig('/nvme/scratch/work/tharvey/EXPANSE/plots/sfh_comparison_no_outshined.pdf', dpi = 200, facecolor = 'white', bbox_inches = 'tight')


In [ ]:

resolved_runs = ['CNST_SFH_RESOLVED', 'CNST_SFH_RESOLVED_NOMIN', 'CNST_SFH_RESOLVED_VORONOI']
resolved_run_names = ['Resolved (piXedfit)', 'Resolved (piXedfit, no min)', 'Resolved (voronoi)']
# nice color palette with distinct colors

resolved_cmap = 'cmr.apple'
resolved_colors = take_cmap_colors(resolved_cmap, len(resolved_runs), cmap_range=(0, 0.8))
resolved_colors = ['mediumseagreen', 'gold', 'orangered']
resolved_color_dict = dict(zip(resolved_runs, resolved_colors))

integrated_runs = ['photoz_lognorm', 'photoz_delayed', 'photoz_continuity', 'photoz_cnst', 'photoz_continuity_bursty']
integrated_run_names = ['Lognorm SFH', r'Delayed-$\tau$ SFH', 'Continuity SFH', 'Constant SFH', 'Bursty SFH']
# uniuqe cmap - not viridis

resolved_masses = {}
for galaxy in galaxies:
    vals = []
    for resolved_run in resolved_runs:
        if galaxy.resolved_mass is not None and resolved_run in galaxy.resolved_mass.keys():
            val = galaxy.resolved_mass[resolved_run][1]
            vals.append(val)
    if len(vals) > 0:
        resolved_masses[galaxy.galaxy_id] = np.nanmax(vals)

integrated_masses = {}
for galaxy in galaxies:
    vals = []
    for integrated_run in integrated_runs:
        if integrated_run in galaxy.sed_fitting_table['bagpipes'].keys():
            if 'TOTAL_BIN' in galaxy.sed_fitting_table['bagpipes'][integrated_run]['#ID']:
                val = galaxy.sed_fitting_table['bagpipes'][integrated_run]['stellar_mass_50'][0]
                vals.append(val)
    if len(vals) > 0:
        integrated_masses[galaxy.galaxy_id] = np.nanmin(vals)

counter = 0

ids = []
# Look for IDs where max resolved mass is close to (within 0.1 dex) of min integrated mass
for galaxy_id, resolved_mass in resolved_masses.items():
    if (integrated_masses[galaxy_id] - resolved_mass) > -0.1 and resolved_mass < 9:
        print(galaxy_id, resolved_mass, integrated_masses[galaxy_id])
        counter += 1
        ids.append(galaxy_id)

print(counter)




In [ ]:
print(ids)

In [ ]:
control_galaxies_all = galaxies.filter_IDs(ids)

In [ ]:
for galaxy in control_galaxies_all:
    galaxy.pixel_by_pixel_galaxy_region(snr_req=2, band_req='all_wide_nobreak', mask='detection')
    galaxy.pixel_by_pixel_binmap(galaxy.gal_region['SNR_2_all_wide_nobreak'])

In [ ]:
nbins = 0
for galaxy in control_galaxies_all:
    nbins += galaxy.get_number_of_bins(binmap_type="pixel_by_pixel")

print(nbins)